In [1]:
%reload_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
import scipy.stats as st
import statsmodels.formula.api as smf
import statsmodels.api as sm
from firth import firthLogit

In [39]:
# Load data
df_choice = pd.read_csv('choice_data.csv',sep=',',index_col=0)

df_choice['part_control'] = (df_choice['group_value'] == False)*df_choice['part_value']
df_choice['part_treatment'] = (df_choice['group_value'] == True)*df_choice['part_value']
 
sum_choice_seq = df_choice.groupby('worker_id')['choice_value'].sum().to_frame()
analysis_workers = sum_choice_seq[(sum_choice_seq['choice_value'] != 24) & (sum_choice_seq['choice_value'] != 0)].index
# df_analysis = df_choice[df_choice['worker_id'].isin(analysis_workers)]
df_analysis = df_choice

# Create dummy variables
worker_dummies = pd.get_dummies(df_analysis['worker_id'], prefix='worker')
front_ratio_dummies = pd.get_dummies(df_analysis['front_ratio'], prefix='front_ratio')
single_amount_dummies = pd.get_dummies(df_analysis['single_amount'], prefix='single_amount')
df_analysis = pd.concat([df_analysis,worker_dummies,front_ratio_dummies,single_amount_dummies],axis=1)

cols_worker = [i for i in worker_dummies if i!= worker_dummies.columns[0]]
cols_front_ratio = [i for i in front_ratio_dummies if i!= front_ratio_dummies.columns[0]]
cols_single_amount = [i for i in single_amount_dummies if i!= single_amount_dummies.columns[0]]

# Covert boolean vairables to numerical variables
bool_cols = df_analysis.select_dtypes(include=['bool']).columns
df_analysis[bool_cols] = df_analysis[bool_cols].astype(int)

# Create regression sample
x_cols = ['group_value','part_control','part_treatment']
y = df_analysis['choice_value']
X = sm.add_constant(df_analysis[x_cols + cols_worker + cols_front_ratio + cols_single_amount])

In [40]:
firth_reg_1 = firthLogit(y,X)
firth_reg_1.fit()

c:\Users\zarkwang\OneDrive\Discounting_Experiment_ZWang\experiment_3\firth.py:14: RuntimeWarning: invalid value encountered in log
  return -(logit.loglike(beta) + 0.5*np.log(np.linalg.det(-logit.hessian(beta))))


iteration: 0 , LL= nan


c:\Users\zarkwang\OneDrive\Discounting_Experiment_ZWang\experiment_3\firth.py:14: RuntimeWarning: invalid value encountered in log
  return -(logit.loglike(beta) + 0.5*np.log(np.linalg.det(-logit.hessian(beta))))


iteration: 1 , LL= nan


c:\Users\zarkwang\OneDrive\Discounting_Experiment_ZWang\experiment_3\firth.py:14: RuntimeWarning: invalid value encountered in log
  return -(logit.loglike(beta) + 0.5*np.log(np.linalg.det(-logit.hessian(beta))))


...

iteration: 8 , LL= nan


c:\Users\zarkwang\OneDrive\Discounting_Experiment_ZWang\experiment_3\firth.py:14: RuntimeWarning: invalid value encountered in log
  return -(logit.loglike(beta) + 0.5*np.log(np.linalg.det(-logit.hessian(beta))))


iteration: 9 , LL= nan


c:\Users\zarkwang\OneDrive\Discounting_Experiment_ZWang\experiment_3\firth.py:14: RuntimeWarning: invalid value encountered in log
  return -(logit.loglike(beta) + 0.5*np.log(np.linalg.det(-logit.hessian(beta))))


iteration: 10 , LL= 1936.0007189386079


In [48]:
wald_result_1 = firth_reg_1.wald()
wald_coef_result_1 = wald_result_1[wald_result_1['var_name'].isin(x_cols)]
wald_coef_result_1.to_csv('firth_reg_result.csv')

Confidence level:  0.95


In [37]:
df_analysis['predict_choice'] = (firth_reg_1.predict() > 0.5).astype(int)
# df_analysis['predict_choice'] = firth_reg_1.predict()
df_analysis['i_group_choice'] = df_analysis['predict_choice']*df_analysis['group_value']
df_analysis['i_control_choice'] = df_analysis['predict_choice']*df_analysis['part_control']
df_analysis['i_treat_choice'] = df_analysis['predict_choice']*df_analysis['part_treatment']

outlier_threshold = df_analysis['response_time'].quantile(0.995)
df_response = df_analysis[df_analysis['response_time'] < outlier_threshold]

In [38]:
y = df_response['response_time']
x_cols_new = x_cols + ['predict_choice','i_group_choice','i_control_choice','i_treat_choice']
X = sm.add_constant(df_response[x_cols_new + cols_worker + cols_front_ratio + cols_single_amount])

mod_2nd = sm.OLS(y,X).fit()
mod_2nd.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          response_time   R-squared:                       0.381
Model:                            OLS   Adj. R-squared:                  0.352
Method:                 Least Squares   F-statistic:                     13.44
Date:                Thu, 11 Apr 2024   Prob (F-statistic):               0.00
Time:                        00:52:49   Log-Likelihood:                -63071.
No. Observations:                7020   AIC:                         1.268e+05
Df Residuals:                    6712   BIC:                         1.289e+05
Df Model:                         307                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const              3235.5350    110.727     29.221      0.000    3018.474    3452.596
group_value       -1931.9570    432.887     -4.463      0.000   -2780.552   -1083.362
part_control       -110.5946     89.528     -1.235      0.217    -286.098      64.909
part_treatment      318.0650     94.931      3.350      0.001     131.970     504.160
predict_choice      657.9102    180.557      3.644      0.000     303.961    1011.859
i_group_choice     -675.6533    230.934     -2.926      0.003   -1128.358    -222.949
i_control_choice   -253.3988    134.540     -1.883      0.060    -517.141      10.343
i_treat_choice      -55.4425    135.159     -0.410      0.682    -320.396     209.511
worker_481485      2052.2118    577.059      3.556      0.000     920.992    3183.432
worker_481487      1457.0833    569.909      2.557      0.011     339.880    2574.287
worker_481489      2787.8131    570.059      4.890      0.000    1670.316    3905.310
worker_481491      1154.2083    569.909      2.025      0.043      37.005    2271.412
worker_481493      1451.1189    587.158      2.471      0.013     300.103    2602.135
worker_481497       517.2842    593.225      0.872      0.383    -645.625    1680.194
worker_481501      2372.1535    575.032      4.125      0.000    1244.907    3499.400
worker_481502      3149.4293    425.684      7.399      0.000    2314.953    3983.905
worker_481503     -1554.0941    412.366     -3.769      0.000   -2362.463    -745.725
worker_481504       356.7834    408.238      0.874      0.382    -443.492    1157.059
worker_481505       577.7649    404.227      1.429      0.153    -214.649    1370.179
worker_481506       -58.0941    412.366     -0.141      0.888    -866.463     750.275
worker_481508     -1492.3416    408.238     -3.656      0.000   -2292.617    -692.066
worker_481509      -575.9691    412.366     -1.397      0.163   -1384.338     232.400
worker_481510      -514.6333    408.238     -1.261      0.207   -1314.908     285.642
worker_481511     -1092.4879    407.062     -2.684      0.007   -1890.459    -294.517
worker_481512      -787.0916    408.238     -1.928      0.054   -1587.367      13.184
worker_481513       609.9583    569.909      1.070      0.285    -507.245    1727.162
worker_481515      -107.5894    587.158     -0.183      0.855   -1258.605    1043.426
worker_481516       348.1464    570.059      0.611      0.541    -769.351    1465.644
worker_481517      1261.1642    575.868      2.190      0.029     132.280    2390.048
worker_481521      2007.9167    569.909      3.523      0.000     890.713    3125.120
worker_481522        96.7083    569.909      0.170      0.865   -1020.495    1213.912
worker_481523       244.1439    577.757      0.423      0.673    -888.444    1376.732
worker_481524       181.2767    576.890      0.314      0.753    -949.612

In [56]:
# Load data
df_rabbit = pd.read_csv('rabbit_data.csv',sep=',',index_col=0)

df_rabbit['part_control'] = (df_rabbit['group_value'] == False)*df_rabbit['part_value']
df_rabbit['part_treatment'] = (df_rabbit['group_value'] == True)*df_rabbit['part_value']
 
# df_analysis = df_choice[df_choice['worker_id'].isin(analysis_workers)]
df_analysis = df_rabbit

# Create dummy variables
worker_dummies = pd.get_dummies(df_analysis['worker_id'], prefix='worker')
front_amount_dummies = pd.get_dummies(df_analysis['front_amount'], prefix='front_amount')
single_amount_dummies = pd.get_dummies(df_analysis['single_amount'], prefix='single_amount')
diff_amount_dummies = pd.get_dummies(df_analysis['diff_amount'], prefix='diff_amount')
df_analysis = pd.concat([df_analysis,worker_dummies,front_amount_dummies,single_amount_dummies,diff_amount_dummies],axis=1)

cols_worker = [i for i in worker_dummies if i!= worker_dummies.columns[0]]
cols_front_amount = [i for i in front_amount_dummies if i!= front_amount_dummies.columns[0]]
cols_single_amount = [i for i in single_amount_dummies if i!= single_amount_dummies.columns[0]]
cols_diff_amount = [i for i in diff_amount_dummies if i!= diff_amount_dummies.columns[0]]

# Covert boolean vairables to numerical variables
bool_cols = df_analysis.select_dtypes(include=['bool']).columns
df_analysis[bool_cols] = df_analysis[bool_cols].astype(int)

# Create regression sample
x_cols = ['group_value','part_control','part_treatment']
y = df_analysis['choice_value']
X = sm.add_constant(df_analysis[x_cols + cols_diff_amount + cols_front_amount + cols_single_amount + cols_worker])

In [57]:
df_rabbit.groupby(['group','part','diff_amount'])['choice_correct'].mean()

group           part         diff_amount
full-exposure   no question  -1             1.000000
                              1             0.976905
                question     -1             0.995227
                              1             0.995506
limit-exposure  no question  -1             0.974537
                              1             0.909910
                question     -1             0.984234
                              1             0.923611
Name: choice_correct, dtype: float64

In [58]:
reg_2 = sm.Logit(y,X).fit()
reg_2.summary()

         Current function value: 0.071438
         Iterations: 35


d:\Python\lib\site-packages\statsmodels\base\model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:           choice_value   No. Observations:                 3504
Model:                          Logit   Df Residuals:                     3204
Method:                           MLE   Df Model:                          299
Date:                Thu, 11 Apr 2024   Pseudo R-squ.:                  0.8968
Time:                        03:19:15   Log-Likelihood:                -250.32
converged:                      False   LL-Null:                       -2426.5
Covariance Type:            nonrobust   LLR p-value:                     0.000
===================================================================================
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
const              -6.5474   7.62e+06  -8.59e-07      1.000   -1.49e+07    1.49e+07
group_value        -4.3790   7.62e+06  -5.75e-07      1.000   -1.49e+07    1.49e+07
part_control        1.3783      0.571      2.415      0.016       0.260       2.497
part_treatment      0.1516      0.288      0.527      0.598      -0.412       0.716
diff_amount_1      11.7104      0.679     17.258      0.000      10.380      13.040
front_amount_2      0.1832      0.306      0.598      0.550      -0.417       0.783
front_amount_3     -0.9501      0.295     -3.217      0.001      -1.529      -0.371
single_amount_8     0.1921      0.240      0.801      0.423      -0.278       0.662
worker_481485       5.1715      5.171      1.000      0.317      -4.963      15.306
worker_481487       1.0499      1.438      0.730      0.465      -1.769       3.869
worker_481489       5.1721      5.077      1.019      0.308      -4.778      15.122
worker_481491       0.9986      1.430      0.698      0.485      -1.805       3.802
worker_481493       1.0368      1.437      0.721      0.471      -1.781       3.854
worker_481497       5.1700      5.161      1.002      0.316      -4.945      15.285
worker_481501       5.1726      5.129      1.009      0.313      -4.880      15.225
worker_481502      -0.0161   7.62e+06  -2.12e-09      1.000   -1.49e+07    1.49e+07
worker_481503      -4.2002   7.62e+06  -5.51e-07      1.000   -1.49e+07    1.49e+07
worker_481504       0.1300   7.62e+06   1.71e-08      1.000   -1.49e+07    1.49e+07
worker_481505       0.1801   7.62e+06   2.36e-08      1.000   -1.49e+07    1.49e+07
worker_481506       0.2940   7.62e+06   3.86e-08      1.000   -1.49e+07    1.49e+07
worker_481508       0.2283   7.62e+06      3e-08      1.000   -1.49e+07    1.49e+07
worker_481509       0.1332   7.62e+06   1.75e-08      1.000   -1.49e+07    1.49e+07
worker_481510       0.2476   7.62e+06   3.25e-08      1.000   -1.49e+07    1.49e+07
worker_481511       0.0895   7.62e+06   1.18e-08      1.000   -1.49e+07    1.49e+07
worker_481512       0.2613   7.62e+06   3.43e-08      1.000   -1.49e+07    1.49e+07
worker_481513       5.1730      5.291      0.978      0.328      -5.197      15.542
worker_481515       9.3068      1.571      5.925      0.000       6.228      12.385
worker_481516       1.0179      1.431      0.711      0.477      -1.788       3.823
worker_481517       5.1673      5.181      0.997      0.319      -4.987      15.322
worker_481521       5.1714      5.170      1.000      0.317      -4.963      15.305
worker_481522       5.1754      5.234      0.989      0.323      -5.083      15.434
worker_481523       1.0313      1.436      0.718      0.473      -1.784       3.846
worker_481524       0.0319      1.261      0.025      0.980      -2.439       2.503
worker_481525      -0.7318      1.226     -0.597      0.550      -3.134       1.670
worker_481526       5.1614      5.108      1.011      0.312      -4.849      15.172
worker_481527       0.0777   7.62e+06   1.02e-08 

In [61]:
df_analysis['predict_choice'] = (reg_2.predict() > 0.5).astype(int)
# df_analysis['predict_choice'] = firth_reg_1.predict()
df_analysis['i_group_choice'] = df_analysis['predict_choice']*df_analysis['group_value']
df_analysis['i_control_choice'] = df_analysis['predict_choice']*df_analysis['part_control']
df_analysis['i_treat_choice'] = df_analysis['predict_choice']*df_analysis['part_treatment']

outlier_threshold = df_analysis['response_time'].quantile(0.995)
df_response = df_analysis[df_analysis['response_time'] < outlier_threshold]

y = df_response['response_time']
x_cols_new = x_cols + ['predict_choice','i_group_choice','i_control_choice','i_treat_choice']
X = sm.add_constant(df_response[x_cols_new + cols_diff_amount + cols_front_amount + cols_single_amount + cols_worker])

mod_2nd = sm.OLS(y,X).fit()
mod_2nd.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          response_time   R-squared:                       0.601
Model:                            OLS   Adj. R-squared:                  0.563
Method:                 Least Squares   F-statistic:                     15.83
Date:                Thu, 11 Apr 2024   Prob (F-statistic):               0.00
Time:                        03:22:08   Log-Likelihood:                -30920.
No. Observations:                3486   AIC:                         6.245e+04
Df Residuals:                    3182   BIC:                         6.432e+04
Df Model:                         303                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const             4050.4172     99.417     40.742      0.000    3855.489    4245.346
group_value      -2452.5914    528.635     -4.639      0.000   -3489.091   -1416.092
part_control       782.1077    125.022      6.256      0.000     536.975    1027.240
part_treatment     788.9140    122.560      6.437      0.000     548.610    1029.218
predict_choice     694.5596    420.615      1.651      0.099    -130.145    1519.264
i_group_choice   -1190.5711    176.118     -6.760      0.000   -1535.887    -845.256
i_control_choice  -129.9164    180.597     -0.719      0.472    -484.015     224.182
i_treat_choice     317.3559    179.389      1.769      0.077     -34.375     669.087
diff_amount_1      915.1617    403.728      2.267      0.023     123.568    1706.755
front_amount_2    -233.1996     74.743     -3.120      0.002    -379.750     -86.650
front_amount_3    -478.1115     75.032     -6.372      0.000    -625.228    -330.995
single_amount_8    542.9783     61.143      8.880      0.000     423.094     662.863
worker_481485     1904.2510    736.259      2.586      0.010     460.661    3347.841
worker_481487     3144.1525    753.626      4.172      0.000    1666.510    4621.795
worker_481489     2124.5306    736.573      2.884      0.004     680.325    3568.737
worker_481491     1389.3343    736.259      1.887      0.059     -54.256    2832.924
worker_481493     3019.5306    736.573      4.099      0.000    1575.325    4463.737
worker_481497      104.0010    736.259      0.141      0.888   -1339.589    1547.591
worker_481501     2578.1139    736.573      3.500      0.000    1133.908    4022.320
worker_481502     3860.0406    541.285      7.131      0.000    2798.738    4921.344
worker_481503    -1854.7450    518.519     -3.577      0.000   -2871.410    -838.080
worker_481504    -1441.3144    518.618     -2.779      0.005   -2458.175    -424.454
worker_481505     2564.8453    518.350      4.948      0.000    1548.511    3581.179
worker_481506    -1456.0784    518.519     -2.808      0.005   -2472.743    -439.414
worker_481508     -473.5784    518.519     -0.913      0.361   -1490.243     543.086
worker_481509     -389.6617    518.519     -0.751      0.452   -1406.326     627.003
worker_481510     -706.0784    518.519     -1.362      0.173   -1722.743     310.586
worker_481511    -1588.4047    518.350     -3.064      0.002   -2604.739    -572.071
worker_481512     -109.4047    518.350     -0.211      0.833   -1125.739     906.929
worker_481513      -91.0287    736.248     -0.124      0.902   -1534.598    1352.541
worker_481515      -24.0527    736.573     -0.033      0.974   -1468.259    1420.153
worker_481516     -501.1361    736.573     -0.680      0.496   -1945.342     943.070
worker_481517     1582.5843    736.259      2.149      0.032     138.994    3026.174
worker_481521     2649.

In [2]:
# Load data
df_choice = pd.read_csv('intertemporal_choice_obs.csv',sep=',',index_col=0)

# Create dummy variables
pid_dummies = pd.get_dummies(df_choice['pid'], prefix='pid')
rw_dummies = pd.get_dummies(df_choice['b_fixed_rw'], prefix='factor_fixed_rw')
delay_dummies = pd.get_dummies(df_choice['b_delay'], prefix='factor_delay')
df_choice = pd.concat([df_choice,pid_dummies,rw_dummies,delay_dummies],axis=1)

cols_pid = [i for i in df_choice.columns if 'pid_' in i and i!= 'pid_1']

# Create interaction terms
factor_cols = [col for col in df_choice.columns if col.startswith('factor_fixed_rw') or col.startswith('factor_delay')]
for factor_col in factor_cols:
    factor_name = factor_col.replace('factor_', '')
    interaction_col = f'I_vary_{factor_name}'
    df_choice[interaction_col] = df_choice[factor_col] * df_choice['b_vary_rw']

# Covert boolean vairables to numerical variables
bool_cols = df_choice.select_dtypes(include=['bool']).columns
df_choice[bool_cols] = df_choice[bool_cols].astype(int)

# Create regression sample for Immed_Rw_Vary
x_cols1 = ['a_rw','b_vary_rw',
           'factor_fixed_rw_7','factor_fixed_rw_9','factor_delay_9','factor_delay_18',
           'I_vary_fixed_rw_7','I_vary_fixed_rw_9','I_vary_delay_9','I_vary_delay_18']

df_choice_immed = df_choice[df_choice['cond'] == 'Immed_Rw_Vary']
y1 = df_choice_immed['choice']
X1 = sm.add_constant(df_choice_immed[x_cols1 + cols_pid])

# Create sample for Delayed_Rw_Vary
x_cols2 = ['a_rw','b_vary_rw',
           'factor_fixed_rw_7','factor_fixed_rw_9',
           'I_vary_fixed_rw_7','I_vary_fixed_rw_9']

df_choice_delayed = df_choice[df_choice['cond'] == 'Delayed_Rw_Vary']
y2 = df_choice_delayed['choice']
X2 = sm.add_constant(df_choice_delayed[x_cols2 + cols_pid])

In [3]:
firth_reg_1 = firthLogit(y1,X1)
firth_reg_1.fit()

iteration: 0 , LL= 12751.908641337992
iteration: 1 , LL= 5848.334232711649
iteration: 2 , LL= 4266.579633452862
iteration: 3 , LL= 3606.98158435943
iteration: 4 , LL= 3389.758368902345
iteration: 5 , LL= 3353.1689375316205
iteration: 6 , LL= 3351.5572946751117
iteration: 7 , LL= 3351.5500972578798
iteration: 8 , LL= 3351.5500932456757
iteration: 9 , LL= 3351.5500932438845
iteration: 10 , LL= 3351.550093243884


In [4]:
wald_result_1 = firth_reg_1.wald()
wald_coef_result_1 = wald_result_1[wald_result_1['var_name'].isin(x_cols1)]
wald_coef_result_1.to_csv('firth_result_immed.csv')

Confidence level:  0.95


In [5]:
df_choice_immed_q = df_choice_immed[['a_rw','b_fixed_rw','b_vary_rw','b_delay']]
df_choice_immed_q['pred_firth'] = firth_reg_1.predict()
pred_firth_immed = df_choice_immed_q.groupby(['a_rw','b_fixed_rw','b_vary_rw','b_delay']).mean('pred_firth')
pred_firth_immed.reset_index().to_csv('firth_pred_immed.csv')

C:\Users\zarkwang\AppData\Local\Temp\ipykernel_1368\2395919629.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_choice_immed_q['pred_firth'] = firth_reg_1.predict()


In [6]:
firth_reg_2 = firthLogit(y2,X2)
firth_reg_2.fit()


iteration: 0 , LL= 6293.7021291342935
iteration: 1 , LL= 2618.0610249924903
iteration: 2 , LL= 1691.1154516442043
iteration: 3 , LL= 1224.3363985323476
iteration: 4 , LL= 992.4025445968265
iteration: 5 , LL= 899.3022986420766
iteration: 6 , LL= 876.6519023539877
iteration: 7 , LL= 874.5239159401027
iteration: 8 , LL= 874.4760096565784
iteration: 9 , LL= 874.4756705870433
iteration: 10 , LL= 874.4756687663141
iteration: 11 , LL= 874.475668756779
iteration: 12 , LL= 874.475668756729
iteration: 13 , LL= 874.4756687567289


In [7]:
wald_result_2 = firth_reg_2.wald()
wald_coef_result_2 = wald_result_2[wald_result_2['var_name'].isin(x_cols2)]
wald_coef_result_2.to_csv('firth_result_delayed.csv')

Confidence level:  0.95


In [8]:
df_choice_delayed_q = df_choice_delayed[['a_rw','b_fixed_rw','b_vary_rw','b_delay']]
df_choice_delayed_q['pred_firth'] = firth_reg_2.predict()
pred_firth_delayed = df_choice_delayed_q.groupby(['a_rw','b_fixed_rw','b_vary_rw','b_delay']).mean('pred_firth')
pred_firth_delayed.reset_index().to_csv('firth_pred_delayed.csv')

C:\Users\zarkwang\AppData\Local\Temp\ipykernel_1368\2143971205.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_choice_delayed_q['pred_firth'] = firth_reg_2.predict()
